In [7]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

Creating list of filepaths to process original event csv data files

In [8]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

c:\Users\Leon\Desktop\ETL_Cassandra


Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [9]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file     
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

NameError: name 'file_path_list' is not defined

In [ ]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

Creating a Cluster

In [ ]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1']) # Cassandra locally installed on the workspace

# To establish connection and begin executing queries, need a session
session = cluster.connect()

Create Keyspace

In [ ]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

Set Keyspace

In [ ]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.
Create queries to ask the following three questions of the data
1. Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4
2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [ ]:
#Here we are going to model our table after the following query
#query = "SELECT * FROM table WHERE sessionid =338 and WHERE itemInSession = 4"
# as we see sessionid and iteminsession uniquely identifies our results
# we therefore are going to use them as our primary key

query = "CREATE TABLE IF NOT EXISTS artist_song_length (\
        sessionid int,\
        itemInSession int,\
        artist text,\
        song_name text,\
        song_length float,\
        ,PRIMARY KEY(sessionid,itemInSession))"

try:
    session.execute(query)
except Exception as e:
    print(e)  

In [ ]:
# loading data into artist_song_length
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_song_length (sessionid, itemInSession,artist, song_name,song_length)"
        query = query + "VALUES (%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[8]),int(line[3]),line[0], line[9],float(line[5])))

Do a SELECT to verify that the data have been inserted into each table

In [ ]:
## testing our table artist_song_length

query  = "SELECT artist,song_name, song_length FROM artist_song_length WHERE sessionid = 338 AND itemInSession = 4 "
try:
    rows=session.execute(query)
except Exception as e:
        print(e)
        
for row in rows:
    print("Artist:",row.artist,"Song Name:", row.song_name,"Length:",row.song_length)

In [ ]:
#Here we want to model our table after the following query
#query = "SELECT artist_name, song_name, user_firstname, user_lastname FROM table2 WHERE userid =10 and sessionid =182"
# We are going to use userid and sessionid as primary key as this obviously identifies our values we ask for
query ="CREATE TABLE IF NOT EXISTS artist_song_user (itemInSession int,\
            userId int,\
            sessionId int,\
            artist text,\
            song_name text,\
            song_length float,\
            user_firstname text,\
            user_lastname text,\
            PRIMARY KEY((itemInSession,userId,sessionId)))"

try:
    session.execute(query)
except Exception as e:
    print(e) 

In [ ]:

with open(file, encoding = 'utf8') as f:
     csvreader = csv.reader(f)
     next(csvreader) # skip header
     for line in csvreader:
         query = "INSERT INTO artist_song_user (itemInSession,userId,sessionID,artist, song_name,song_length,user_firstname,user_lastname)"
         query = query + "VALUES (%s,%s,%s,%s,%s,%s,%s,%s)"
         session.execute(query, (int(line[3]),int(line[10]),int(line[8]),line[0],line[9],float(line[5]),line[1],line[4]))
            
test_query = "SELECT artist, song_name, user_firstname, user_lastname FROM artist_song_user WHERE userid =10 \
                AND sessionid =182"

try:
     rows = session.execute(test_query)
except Exception as e:
     print(e)

for row in rows:
     print("Artist:",row.artist,"Song", row.song_name, "First Name:", row.user_firstname,"last name:",row.user_lastname)

In [ ]:
#creating our table user_name to get the results for the following query
#query = "SELECT user_firstname, user_lastname FROM user_name WHERE song_name = 'All Hands Against' "
#here our primary key has to consist of the song_name and an identifier for the user
#to only user one additional key we can choose userid

query = "CREATE TABLE IF NOT EXISTS user_name (song_name text,userid int, artist text, user_firstname text,user_lastname text,\
        sessionid int, PRIMARY KEY(song_name,userid))"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [ ]:
# Inserting data into the user_name table as before

with open(file, encoding = 'utf8') as f:
     csvreader = csv.reader(f)
     next(csvreader) # skip header
     for line in csvreader:
         query = "INSERT INTO user_name (song_name,\
                                         userid,\
                                        artist,\
                                        user_firstname,\
                                        user_lastname,\
                                         sessionId)"
         query = query + "VALUES (%s,%s,%s,%s,%s,%s)"
        
         session.execute(query, (line[9],int(line[10]), line[0],line[1],line[4],int(line[8])))

In [ ]:
#testing our table with our query
test_query = "SELECT user_firstname,user_lastname FROM user_name WHERE song_name = 'All Hands Against' "
try:
    rows = session.execute(test_query)
except Exception as e:
    print(e)
for row in rows:
    print(row)

In [ ]:
#dropping all our tables
query1 = "DROP TABLE artist_song_length"
query2 = "DROP TABLE artist_song_user"
query3 = "DROP TABLE user_name"

try:
    session.execute(query1)
except Exception as e:
    print(e)

try:
    session.execute(query2)
except Exception as e:
    print(e)
try:
    session.execute(query3)
except Exception as e:
    print(e)

Close the session and cluster connection

In [ ]:
session.shutdown()
cluster.shutdown()